# Instalando dependências

In [195]:
%pip install -r dependencias/requirements.txt

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# Importando bibliotecas

In [196]:
import pandas as pd
import numpy as np
import random

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

# Coleta de dados

In [197]:
cestas = pd.read_csv("bases/cestas.csv")
clientes = pd.read_csv('bases/clientes.csv')
produtos = pd.read_csv('bases/produtos.csv')

# Produtos mais vendidos

In [198]:
faturamento_por_produto = cestas.groupby('item_id').agg({'price': 'sum'}).reset_index()
top3_faturamento = faturamento_por_produto.sort_values('price', ascending=False).head(3)
top3_faturamento['rec_source'] = 'top products'

In [199]:
top3_faturamento

,item_id,price,rec_source
62,item_2,7407614.61,top products
68,item_25,7225204.80,top products
76,item_32,5993105.58,top products


# Definição de funções

In [200]:
def prepare_data_apriori(df):

    transactions = df.groupby('basket_id')['item_id'].apply(list).tolist()
    
    encoder = TransactionEncoder()
    encoder_fitted = encoder.fit(transactions).transform(transactions)
    encoded_df = pd.DataFrame(encoder_fitted, columns = encoder.columns_)

    return encoded_df

In [201]:
def compute_association_rules(encoded_df, min_support=0.01, top=20):

    apriori_associations = apriori(
        encoded_df,
        min_support=min_support,
        use_colnames=True
    )

    if len(apriori_associations) == 0:
        return pd.DataFrame()

    rules = association_rules(
        apriori_associations,
        metric="lift",
        min_threshold=1
    )

    if len(rules) == 0:
        return pd.DataFrame()

    rules = rules.drop_duplicates(subset=['antecedents', 'consequents'])

    rules = rules.sort_values(
        by=["lift", "confidence", "support"],
        ascending=False
    ).head(top)

    # Salvar as regras do Apriori em um CSV
    rules.to_csv('regras_apriori.csv', index=False)
    print("Arquivo regras_apriori.csv gerado com sucesso.")
    return rules

In [202]:
def recommend_for_users(df_baskets, rules):

    df_baskets["timestamp"] = pd.to_datetime(df_baskets["timestamp"], errors="coerce")

    df_mes = df_baskets[
        (df_baskets["timestamp"].dt.year == 2025) &
        (df_baskets["timestamp"].dt.month == 12)
    ].copy()

    recomendacoes = []

    for user_id, group in df_mes.groupby("user_id"):

        itens_user = set(group["item_id"].unique())

        melhores_por_consequente = {}

        for _, rule in rules.iterrows():

            antecedent = set(rule["antecedents"])
            consequent = set(rule["consequents"])

            if not antecedent.issubset(itens_user):
                continue

            novos_itens = list(consequent - itens_user)

            if len(novos_itens) == 0:
                continue

            for item_rec in novos_itens:

                if item_rec in melhores_por_consequente:
                    if rule["lift"] > melhores_por_consequente[item_rec]["lift"]:
                        melhores_por_consequente[item_rec] = {
                            "user_id": user_id,
                            "antecedent": tuple(antecedent),
                            "consequent": item_rec,
                            "support": rule["support"],
                            "confidence": rule["confidence"],
                            "lift": rule["lift"]
                        }
                else:
                    melhores_por_consequente[item_rec] = {
                        "user_id": user_id,
                        "antecedent": tuple(antecedent),
                        "consequent": item_rec,
                        "support": rule["support"],
                        "confidence": rule["confidence"],
                        "lift": rule["lift"]
                    }

        recomendacoes.extend(melhores_por_consequente.values())

    df_rec = pd.DataFrame(recomendacoes)

    if len(df_rec) > 0:
        df_rec = df_rec.sort_values(
            by=["user_id", "lift", "confidence", "support"],
            ascending=[True, False, False, False]
        )

    return df_rec

In [203]:
def completar_recomendacoes(grupo):
    grupo = grupo.copy()

    recomendações_validas = grupo[grupo['rec'].notna()].head(3)
    n_validas = len(recomendações_validas)
    
    if n_validas >= 3:
        return recomendações_validas.head(3)

    linhas_necessarias = 3 - n_validas
    produtos_top = top3_faturamento.head(linhas_necessarias).copy()

    for _, produto in produtos_top.iterrows():
        nova_linha = pd.DataFrame({
            'user_id': [grupo['user_id'].iloc[0]],
            'rec': [produto['item_id']],
            'source_rec': ['top products']
        })
        recomendações_validas = pd.concat([recomendações_validas, nova_linha], ignore_index=True)
    
    return recomendações_validas

In [204]:
encoded_df_cestas = prepare_data_apriori(cestas)
rules = compute_association_rules(encoded_df_cestas, min_support=0.01, top=30)

Arquivo regras_apriori.csv gerado com sucesso.


In [205]:
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
50,"(item_1, item_2)",(item_5),0.1158,0.1256,0.0182,0.157168,1.251334,0.003656,1.037454,0.227158
55,(item_5),"(item_1, item_2)",0.1256,0.1158,0.0182,0.144904,1.251334,0.003656,1.034036,0.229704
52,"(item_2, item_5)",(item_1),0.0324,0.4708,0.0182,0.561728,1.193136,0.002946,1.207470,0.167293
53,(item_1),"(item_2, item_5)",0.4708,0.0324,0.0182,0.038658,1.193136,0.002946,1.006509,0.305882
51,"(item_1, item_5)",(item_2),0.0598,0.2558,0.0182,0.304348,1.189788,0.002903,1.069787,0.169660


In [206]:
recs = recommend_for_users(cestas, rules)

In [207]:
recs.head()

,user_id,antecedent,consequent,support,confidence,lift
0,user_101,"(item_5,)",item_2,0.0182,0.144904,1.251334
1,user_101,"(item_1,)",item_25,0.0148,0.031436,1.164291
2,user_101,"(item_5,)",item_4,0.0100,0.079618,1.137398
3,user_101,"(item_1,)",item_6,0.0136,0.028887,1.094205
4,user_101,"(item_1,)",item_30,0.0108,0.022940,1.092366


In [208]:
df_merged = pd.merge(recs, clientes, on='user_id', how='outer')
#df_merged['source_rec'] = np.where(df_merged['consequent'].notna(), 'apriori', np.nan)
df_merged['source_rec'] = df_merged['consequent'].apply(
    lambda x: 'apriori' if pd.notna(x) else None
)
df_merged = df_merged[['user_id', 'consequent', 'source_rec']].rename(columns={'consequent': 'rec'})

In [209]:
df_merged

,user_id,rec,source_rec
0,user_101,item_2,apriori
1,user_101,item_25,apriori
2,user_101,item_4,apriori
3,user_101,item_6,apriori
4,user_101,item_30,apriori
...,...,...,...
749,user_494,NaN,None
750,user_495,NaN,None
751,user_496,NaN,None
752,user_498,NaN,None


In [210]:
df_recs_preenchido = df_merged.groupby('user_id').apply(completar_recomendacoes).reset_index(drop=True)

In [211]:
df_recs_preenchido

,user_id,rec,source_rec
0,user_1,item_2,top products
1,user_1,item_25,top products
2,user_1,item_32,top products
3,user_10,item_2,top products
4,user_10,item_25,top products
...,...,...,...
1495,user_98,item_25,top products
1496,user_98,item_32,top products
1497,user_99,item_2,top products
1498,user_99,item_25,top products


In [212]:
df_completo = pd.merge(df_recs_preenchido, clientes, on='user_id', how='left')
df_completo = pd.merge(df_completo, produtos, left_on='rec', right_on='item_id', how='left')

In [213]:
df_output = df_completo[['user_id','nome','documento','telefone','uf','cluster','item_id','item_desc','source_rec']].rename(columns={'item_desc': 'rec_desc', 'item_id': 'rec_id'})

In [214]:
df_output.head()

,user_id,nome,documento,telefone,uf,cluster,rec_id,rec_desc,source_rec
0,user_1,Leonardo Lima,154.171.609-44,(15) 65524-5576,SC,SILVER,item_2,Tiodicarbe,top products
1,user_1,Leonardo Lima,154.171.609-44,(15) 65524-5576,SC,SILVER,item_25,Spiromesifen,top products
2,user_1,Leonardo Lima,154.171.609-44,(15) 65524-5576,SC,SILVER,item_32,Buprofezina,top products
3,user_10,Olivia Rocha,929.861.623-60,(89) 67039-6637,RR,SILVER,item_2,Tiodicarbe,top products
4,user_10,Olivia Rocha,929.861.623-60,(89) 67039-6637,RR,SILVER,item_25,Spiromesifen,top products


In [215]:
df_output.to_csv('recomendacoes.csv', index=False)

## Adicionando informações dos Clientes

In [216]:
df_recomendacoes = pd.read_csv("recomendacoes.csv")

In [217]:
# Remover duplicatas para ter uma lista única de clientes
df_clientes_unicos = df_recomendacoes[['user_id', 'nome', 'documento', 'telefone', 'uf', 'cluster']].drop_duplicates().reset_index(drop=True)

In [218]:
# Listas de componentes de nomes de empresas de agronegócio
prefixos_fazenda = ['Fazenda', 'Sítio', 'Chácara', 'Agropecuária', 'Granja', 'Cooperativa']
nomes_compostos = ['Esperança', 'São João', 'Boa Vista', 'Horizonte', 'Verde', 'Ouro', 'Progresso', 'Lider']
sufixos = ['& Filhos', 'LTDA', 'EIRELI', 'Agro', 'Rural', 'Comércio']

def gerar_nome_empresa(nome_pessoa):
    """Gera um nome de empresa/fazenda mais realista para o agronegócio."""
    
    prefixo = random.choice(prefixos_fazenda)
    nome = random.choice(nomes_compostos)
    
    partes_nome = nome_pessoa.split()
    if len(partes_nome) > 1:
        sobrenome = partes_nome[-1]
        nome_completo = f"{prefixo} {nome} {random.choice(['do', 'da'])} {sobrenome}"
    else:
        nome_completo = f"{prefixo} {nome}"

    if random.random() < 0.2: 
        nome_completo += f" {random.choice(sufixos)}"
        
    return nome_completo.replace('Da Pessoa', '')

In [219]:
# Aplicar a função para criar a coluna de Nome da Empresa
df_clientes_unicos['nome_empresa'] = df_clientes_unicos['nome'].apply(lambda x: gerar_nome_empresa(x))

In [220]:
# Lista de UF e Regiões para mapeamento (pode ser expandida)
regioes_map = {
    'MT': 'Centro-Oeste', 'MS': 'Centro-Oeste', 'GO': 'Centro-Oeste',
    'BA': 'Nordeste', 'PI': 'Nordeste', 'PR': 'Sul', 'SP': 'Sudeste', 'PE': 'Nordeste', 'RR': 'Norte', 'RN': 'Nordeste', 'PA': 'Norte'
}

In [221]:
# Listas realistas para amostragem no agronegócio
cidades_centro_oeste = ['Sorriso', 'Lucas do Rio Verde', 'Campo Grande', 'Rondonópolis', 'Goiânia']
culturas = ['Soja', 'Milho', 'Algodão', 'Cana-de-Açúcar', 'Café', 'Trigo']
tipos_solo = ['Latossolo Vermelho', 'Argissolo', 'Latossolo Amarelo', 'Neossolo', 'Terra Roxa']
pragas = ['Lagarta', 'Ferrugem', 'Percevejos', 'Nematoides', 'Cigarrinha']
safra_principal = ['Out-Mar', 'Set-Fev', 'Nov-Abr', 'Abr-Ago']

In [222]:
def gerar_dados_agronomicos(row):
    """Gera dados realistas e randômicos de agronegócio para um único cliente."""
    
    # 1. Responsável e Email 
    # O nome da pessoa original ('nome' em df_clientes_unicos) é o Responsável
    responsavel = row['nome'] 
    
    # Simula um Email simples baseado no nome da empresa
    email_base = row['nome_empresa'].lower().replace(' ', '').replace('&', '').replace('ltda', '').replace('eireli', '')
    email = f"{email_base}@{random.choice(['agrocorp', 'fazenda', 'ruraltec'])}.com.br"
    
    # 2. Localização
    uf = row['uf']
    regiao = regioes_map.get(uf, 'Outra')
    cidade = random.choice(cidades_centro_oeste) if regiao == 'Centro-Oeste' else f'Cidade {uf}'

    # 3. Dados Agronômicos
    num_culturas = random.randint(1, 2)
    culturas_selecionadas = " / ".join(random.sample(culturas, num_culturas))

    # Área total (ajustada pelo cluster: Diamond deve ter áreas maiores)
    multiplicador = {'DIAMOND': 1.5, 'GOLD': 1.2, 'SILVER': 1.0, 'BRONZE': 0.8}.get(row['cluster'], 1.0)
    area_total = int(np.round(random.uniform(500, 2500) * multiplicador, -2))
    
    praga_comum = " / ".join(random.sample(pragas, 2)) if random.random() < 0.7 else random.choice(pragas)

    return pd.Series({
        'responsavel': responsavel,
        'email': email,
        'cidade': cidade,
        'regiao': regiao,
        'culturas': culturas_selecionadas,
        'area_total': area_total,
        'tipo_solo': random.choice(tipos_solo),
        'praga_comum': praga_comum,
        'safra_principal': random.choice(safra_principal)
    })

In [223]:
# Aplica a função de geração de dados ao DataFrame de clientes únicos
novas_colunas_df = df_clientes_unicos.apply(gerar_dados_agronomicos, axis=1)

In [224]:
# Junta as novas colunas ao DataFrame de clientes
# Move a coluna 'nome' (nome da pessoa) para ser o 'responsavel'
df_clientes = pd.concat([df_clientes_unicos.drop(columns=['nome']), df_clientes_unicos['nome_empresa'], novas_colunas_df], axis=1)

In [225]:
# Renomear a coluna 'nome_empresa' para 'nome', e 'nome' original para 'responsavel'
df_clientes = df_clientes.rename(columns={'nome_empresa': 'nome', 'nome': 'responsavel'})

In [226]:
# Reordena as colunas para seguir o exemplo fornecido pelo usuário e garantir a ordem no Streamlit
colunas_finais = ['user_id', 'nome', 'responsavel', 'documento', 'telefone', 'email', 
                  'cidade', 'uf', 'regiao', 'culturas', 'cluster', 'area_total', 
                  'tipo_solo', 'praga_comum', 'safra_principal']
df_clientes = df_clientes[colunas_finais]

In [227]:
# --- 3. Exibir o resultado final e salvar ---
print("DataFrame Final de Clientes (df_clientes) com dados randômicos de Agronegócio:")
print(df_clientes.head())

DataFrame Final de Clientes (df_clientes) com dados randômicos de Agronegócio:
    user_id                            nome                            nome  \
0    user_1  Agropecuária Boa Vista da Lima  Agropecuária Boa Vista da Lima   
1   user_10          Chácara Verde da Rocha          Chácara Verde da Rocha   
2  user_100      Granja São João do Almeida      Granja São João do Almeida   
3  user_101        Sítio Esperança do Souza        Sítio Esperança do Souza   
4  user_102      Chácara Esperança da Costa      Chácara Esperança da Costa   

     responsavel       documento         telefone  \
0  Leonardo Lima  154.171.609-44  (15) 65524-5576   
1   Olivia Rocha  929.861.623-60  (89) 67039-6637   
2   Igor Almeida  004.664.849-47  (21) 63928-4713   
3  Daniela Souza  556.649.635-30  (48) 35581-2597   
4  Eduardo Costa  233.132.947-39  (60) 69072-1605   

                                        email     cidade  uf    regiao  \
0  agropecuáriaboavistadalima@ruraltec.com.br  Cidade

In [228]:
# Salvar o resultado para uso posterior no Streamlit
df_clientes.to_csv("clientes_df.csv", index=False)
print("\nArquivo 'clientes_df.csv' gerado com sucesso na pasta do projeto.")


Arquivo 'clientes_df.csv' gerado com sucesso na pasta do projeto.
